In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import pandas as pd
import xlsxwriter
import math
from PIL import Image
import numpy as np
from IPython.display import display
import jaydebeapi
import ConfigParser
import sys
import urllib
import requests
pd.options.display.max_columns = None
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import line_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from nltk import sent_tokenize
from nltk import pos_tag, sent_tokenize, wordpunct_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import Binarizer
from wordcloud import WordCloud
from collections import Counter
from palettable.colorbrewer.sequential import Reds_9
import random
from IPython.core.interactiveshell import InteractiveShell
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA

InteractiveShell.ast_node_interactivity = "all"

##https://minimaxir.com/2016/05/wordclouds/
##https://www.kdnuggets.com/2018/04/implementing-deep-learning-methods-feature-engineering-text-data-skip-gram.html

In [64]:
df = pd.read_csv('/Users/exue001c/evagit/amazon_healthcare_review/data/review_cleaned.csv')

In [65]:
df.head()

,Unnamed: 0,helpful_votes,product_id,rating,review,review_date,review_len,title,title_len,one_gram_lema_text,n_gram_lema_text,orig_text,one_gram_lema_text.1,n_gram_lema_text.1,orig_text.1
0,0,0,B07H7Q1DFP,4,Got these in hopes that my hands would not hur...,1553126400000,418,"Very soft, saves the hands and under arms",41,got hope hand would hurt use comfortable hand ...,got_hope hope_hand hand_would would_not not_hu...,Got these in hopes that my hands would not hur...,soft save hand arm,soft_save save_hand hand_arm,"Very soft, saves the hands and under arms"
1,1,0,B07HM7P88N,5,The product came very fast and it is very cush...,1557964800000,94,It does work!,13,product come fast cushy palm hand armpit,product_come come_fast fast_cushy cushy_palm p...,The product came very fast and it is very cush...,work,NaN,It does work!
2,2,0,B07HM7P88N,5,My daughter had to look for support for her cr...,1553385600000,143,My daughter loves it. Best support for your cr...,53,baby save armpit heel hand recover knee surger...,baby_save save_armpit armpit_heel heel_hand ha...,These babies have saved my armpits and the hee...,comfort longer term use,comfort_longer longer_term term_use,Comfort for longer term crutch use
3,3,0,B01IBUD24G,5,"I'm a student in college, and this was super h...",1523577600000,104,and this was super helpful to carry around my ...,71,worked great,worked_great,Worked out great.,good,NaN,All Good.
4,4,3,B01IBUD24G,4,The crutch bag gets five stars for being super...,1500076800000,595,"Great idea and super handy, but recommended wi...",62,purchase daughter surgery leg helpful add comf...,purchase_daughter daughter_surgery surgery_leg...,I purchased these for my daughter when she had...,nice product,nice_product,Very Nice Product


In [66]:
review_list = df.one_gram_lema_text.dropna().to_list()

In [67]:
no_features = 1000

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [68]:
tf_idf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,max_features=no_features)
X_tfidf = tf_idf_vectorizer.fit_transform(review_list)
##tf_idf_norm = normalize(X_tfidf)
#tf_idf_array = tf_idf_norm.toarray()
tfidf_feature = tf_idf_vectorizer.get_feature_names()
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns = tfidf_feature)
tfidf_df.shape

(2478, 1000)

Inverse document frequency is defined as:

𝑖𝑑𝑓(𝑡,𝐷)=𝑙𝑜𝑔𝑁|𝑑∈𝐷:𝑡∈𝑑|

Where 𝑁 is the total number of documents in the corpus, and |𝑑∈𝐷:𝑡∈𝑑| is the number of documents where 𝑡 appears.

TfidfVectorizer and CountVectorizer both are methods for converting text data into vectors as model can process only numerical data.

In CountVectorizer we only count the number of times a word appears in the document which results in biasing in favour of most frequent words. this ends up in ignoring rare words which could have helped is in processing our data more efficiently.

To overcome this , we use TfidfVectorizer .

In TfidfVectorizer we consider overall document weightage of a word. It helps us in dealing with most frequent words. Using it we can penalize them. TfidfVectorizer weights the word counts by a measure of how often they appear in the documents.

In [69]:
vec = CountVectorizer(max_df=0.95, min_df=2,max_features=no_features)
#vec = CountVectorizer(binary=True)
X_tf = vec.fit_transform(review_list)
tf_feature = vec.get_feature_names()
tf_df = pd.DataFrame(X_tf.toarray(), columns = tf_feature)
tf_df.shape

(2478, 1000)

In [70]:
from sklearn.decomposition import LatentDirichletAllocation , NMF

 ## NMF -  relies on linear algebra
 Non-negative Matrix Factorization (NMF)
 
 NMF is able to use tf-idf
 https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html

## LDA - is based on probabilistic graphical modeling 
LDA can only use raw term counts for LDA because it is a probabilistic graphical model


Feed word count

In [71]:
n_topics = 4

In [72]:
# Run NMF
nmf = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(X_tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(X_tf)


/Users/exue001c/miniconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [73]:
def print_top_words(model, feature_names, n_top_words):
    
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print(' ')

In [74]:
print("Topics in LDA model:")
print_top_words(lda, tf_feature, n_top_words)
print("Topics in NMF model:")
print_top_words(nmf, tfidf_feature, n_top_words)

Topics in LDA model:
Topic #0:
recommend highly color like would soft new expect anyone attach
Topic #1:
get use hand go would time cushion week armpit one
Topic #2:
fit great work carry phone well good use bag product
Topic #3:
comfortable make use easy love daughter hand much arm help
 
Topics in NMF model:
Topic #0:
use make easy hand get much love arm well help
Topic #1:
comfortable make much made definitely super easy use daughter soft
Topic #2:
great product work worked price quality value item comfort seller
Topic #3:
fit perfect perfectly well work old good replacement quality like
 


https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/
https://stackabuse.com/implementing-lda-in-python-with-scikit-learn/

https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/#5.-Build-the-Topic-Model
https://medium.com/@sherryqixuan/topic-modeling-and-pyldavis-visualization-86a543e21f58
https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/


In [15]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
p = pyLDAvis.sklearn.prepare(lda, X_lemma, vec)
pyLDAvis.save_html(p,'lda.html')

In [14]:
pyLDAvis.save_html(p,'lda.html')

TypeError: save_html() takes exactly 2 arguments (3 given)

Topic 1 - Review ont the product quality eg., 
Topic 2
Topic 3
Topic 4